In [943]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [944]:
#Define constants of cost storage function
C = 5
K1 = 1

#Function Cost storage based on units
def cost_storage(qt, st):
  n = qt - st

  if(n <= 0): return 0
  return C*n + K1

In [945]:
#Define onstants of penalization function
D = 35
K2 = 5



In [946]:
from scipy.stats import norm

In [947]:
#Density funciton


def f_density(x, mu):  
    aux = norm.pdf(x, mu, SIGMA)
    return aux

In [948]:
#Constants
step = 1

#Loss function
def f_loss(qt,st):
    loss = 0

    #first integral
    rng = np.arange(st, qt+step, step)

    for i in rng:
        loss += cost_storage(i,st)*f_density(i,qt)
    
    

    #second integral
    rng2 = np.arange(0, st+step, step)
    for i in rng2:
        loss += penalization_cost(i,st)*f_density(i,qt)
    

    return loss

In [949]:
#Constants
step_s = 10

#Solve qt
def solve_qt_it(st,it):
    if(st == it):
        return st
    
    rng = np.arange(st, it, step_s)

    min_loss = np.inf
    min_qt = 1000

    for i in rng:
        aux_loss = f_loss(i,st)
        if(aux_loss < min_loss):
            min_loss = aux_loss
            min_qt = i

    return min_qt

In [950]:
#Solve solution
def general_solution(v_it, v_st):
    v_qt = []
    for i in range(0, len(v_it)):
        v_qt.append(solve_qt_it(v_st[i], v_it[i]))
        #print("   ", i)
    
    return v_qt

In [951]:
#Test 1
df_train = pd.read_csv("train.csv")
df_train = df_train.sort_values(by = ["date"])


In [952]:
df_train.tail(5)

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
9952,202318-172725,2023-05-06,202318,172725,12,Goku,163423,15.6,Gaming,0.0,26.0
5228,202318-80934,2023-05-06,202318,80934,3,Conan,151819,13.5,Premium,121.0,89.0
5229,202318-80934,2023-05-06,202318,80934,24,Conan,151819,13.5,Premium,36.0,68.0
15151,202318-242802,2023-05-06,202318,242802,93,Conan,3868,13.3,Premium,5.0,3.0
15453,202318-247737,2023-05-06,202318,247737,24,Arale,10637,13.3,Premium,5.0,1.0


In [953]:
list_id = df_train["product_number"].unique()
list_da = df_train["date"].unique()
list_da = list_da[-100:]
cont = 0

sns.set_theme(style="darkgrid")

for p in list_id:
    it = []
    st = []
    qt = []
    for d in list_da:
        val = df_train[(df_train['product_number']==p) & (df_train['date']==d)]['inventory_units'].sum()
        val2 = df_train[(df_train['product_number']==p) & (df_train['date']==d)]['sales_units'].sum()
        if(val2 > val):
            val2 = val
        it.append(val)
        st.append(val2)
    
    print(cont)
    qt = general_solution(it, st)
    cont += 1

    

    x = np.asarray(list_da, dtype='datetime64[s]')
    sns.lineplot(x = x, y = it, label='Inventory')
    sns.lineplot(x = x, y = st, label='Sales')
    ax = sns.lineplot(x = x, y = qt, label='Planning')
    plt.tick_params(axis='x', labelrotation=60)
    ax.set(xlabel='Date', ylabel='Units')
    plt.legend(title='Inventory planning', loc='upper left')
    plt.show()

    if(cont == 10): break

KeyboardInterrupt: 